In [ ]:
##Avec tensorflow


import tensorflow as tf

tf.reset_default_graph()

inputs=1
output=1
hidden=100

X = tf.placeholder(tf.float32, [None, n_lag, inputs])
Y = tf.placeholder(tf.float32, [None, n_lag, output])

b_cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden,activation=tf.nn.relu)
rnn_output, states = tf.nn.dynamic_rnn(b_cell, X, dtype=tf.float32)

learning_rate = 0.0001

stacked_rnn_output = tf.reshape(rnn_output, [-1,hidden])
stacked_output = tf.layers.dense(stacked_rnn_output, output)
outputs = tf.reshape(stacked_output, [-1,n_lag,output])

loss = tf.reduce_sum(tf.square(outputs-Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [ ]:
TS = np.array(DSvol.y)
horizon = 1 #Equvau window_size-n_lag  de KERAS

x_data = TS[:(len(TS)-(len(TS) % n_lag))]
x_batches = x_data.reshape(-1, n_lag, output)

y_data = TS[1:(len(TS)-(len(TS) % n_lag))+horizon]
y_batches = y_data.reshape(-1, n_lag, output)



In [ ]:
def test_data(series, forecast, n_lag):
    test_x_setup = TS[-(n_lag + forecast):]
    testX = test_x_setup[:n_lag].reshape(-1, n_lag, output)
    testY = TS[-(n_lag):].reshape(-1, n_lag, output)
    return testX, testY

X_test, Y_test = test_data(TS, horizon, n_lag)

In [ ]:
epo = 2000

with tf.Session() as sess:
    init.run()
    for ep in range(epo):
        sess.run(training_op, feed_dict={X: x_batches, Y: y_batches})
        if ep % 100 == 0:
            mse = loss.eval(feed_dict={X: x_batches, Y: y_batches})
            print(ep, "\tMSE:", mse)
            
    y_pred = sess.run(outputs, feed_dict={X: X_test})
    print(y_pred)

In [ ]:
plt.figure(figsize=(20,5))
plt.title("Forecast vs Actual", fontsize=14)
plt.plot(pd.Series(np.ravel(Y_test)), label="Actual")
plt.plot(pd.Series(np.ravel(y_pred)), label="Forecast")
plt.legend(loc="upper left")
plt.xlabel("Time Periods")
plt.show()